In [1]:
import pandas as pd
from arango import ArangoClient

import numpy as np

# connexion à la base de données
client = ArangoClient(hosts="http://localhost:8529")
sys_db = client.db('_system', username='root',password='root')
db = client.db("text", username="root", password="root")

## graphe de similitudes

In [2]:
sentences_ID_df = pd.DataFrame(list(
    db.aql.execute('''
    FOR doc in sentences
    return doc._id
    ''')
    )
                              )
sentences_ID_df

,0
0,sentences/doc0sent0
1,sentences/doc0sent1
2,sentences/doc0sent10
3,sentences/doc0sent100
4,sentences/doc0sent1000
...,...
22946,sentences/doc3sent95
22947,sentences/doc3sent96
22948,sentences/doc3sent97
22949,sentences/doc3sent98


In [3]:
sentences_ID_df.iloc[0,0]

'sentences/doc0sent0'

In [4]:
lemmas_from_sentences = pd.DataFrame(list(db.aql.execute('''for start_vertex in sentences
        for v, e in inbound start_vertex is_from
        filter e.type == 'lemmaToSent'
        collect sent = e._to, lemmas = v.lemma into groups ={
        "sentence" : e._to,
        "lemma" : v.lemma
        }
        return {"sentence":sent,
                "lemma":lemmas}
                '''))).groupby('sentence')['lemma'].apply(' '.join)
lemmas_from_sentences

sentence
sentences/doc0sent0                                                     30
sentences/doc0sent1                                                     -on
sentences/doc0sent10      1(2 1(4 and centre control correct credential ...
sentences/doc0sent1000                                           and social
sentences/doc0sent1001                           and based be been involved
                                                ...                        
sentences/doc3sent93                          cned dynamique employeur fois
sentences/doc3sent94                                                  doute
sentences/doc3sent96                                               fonction
sentences/doc3sent97      activité attribut cadre dela enseignement entr...
sentences/doc3sent98       destin donner douvrier enseigner formation forme
Name: lemma, Length: 18693, dtype: object

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import minmax_scale
import networkx as nx
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='browser'


In [6]:
vectorizer = CountVectorizer(min_df=5)

termDocMatrix  = vectorizer.fit_transform(lemmas_from_sentences)

In [10]:
coOccurenceMatrix = termDocMatrix.T.dot(termDocMatrix)
# retire les liens d'un nodeà lui même dnas la matrice
coOccurenceMatrix

In [ ]:

# construction du graphe
G = nx.from_scipy_sparse_array(coOccurenceMatrix,
                                parallel_edges=False)

# retire les arretes qui connectent un noeud à lui même 

G.remove_edges_from(nx.selfloop_edges(G))



# définition de la position des noeuds par spatialisation fruchterman reingold

FRL = nx.drawing.layout.fruchterman_reingold_layout(G)

# kamada kawai
# KMK = nx.drawing.layout.kamada_kawai_layout(G)

for i in range(0,len(FRL)):
    G.nodes[i]['pos'] = FRL[i]
    


def make_edge(x, y, width,scaledWidth):
    """
    Args:
        x: a tuple of the x from and to, in the form: tuple([x0, x1, None])
        y: a tuple of the y from and to, in the form: tuple([y0, y1, None])
        width: The width of the line

    Returns:
        a Scatter plot which represents a line between the two points given. 
    """
    return  go.Scatter(
                x=x,
                y=y,
                line=dict(width=width,color='#888'),
                hoverinfo='none',
                mode='lines',
                opacity=scaledWidth)




xTupleList = []
yTupleList = []

for ed in G.edges(): 
    xfrom = G.nodes()[ed[0]]['pos'][0]
    yfrom = G.nodes()[ed[0]]['pos'][1]
    
    xto = G.nodes()[ed[1]]['pos'][0]
    yto = G.nodes()[ed[1]]['pos'][1]
    xTupleList.append((xfrom,xto,None))
    yTupleList.append((yfrom,yto,None))

widthList = np.array([G.edges[ed]['weight'] for ed in G.edges()])

scaledWidthList = minmax_scale(widthList)



edge_trace = [make_edge(x,y,w,sw) for x,y,w,sw in zip(xTupleList,yTupleList,widthList*0.5,scaledWidthList)]




node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
    
    
    

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title="Nombre d'occurences",
            xanchor='left',
            titleside='right'
        ),
        line_width=2))





node_trace.marker.color = list(vectorizer.vocabulary_.values())
node_trace.text = list(vectorizer.vocabulary_.keys())


layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)


fig = go.Figure(layout = layout)

# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)# Add node trace
fig.add_trace(node_trace)# Remove legend
fig.update_layout(showlegend = False)


## graphe syntagmatique

sur la base d'un lemme, obtenir :
- les tokens associés
- les relations syntagmatiques de ces tokens
- les tokens à l'issue de ces relations
- la source de ces relations (pour l'instant le document mais se donner l'option d'ajouter d'autres variables à l'avenir)

faire une requête de tous les tokens liés à un lemme avec un groupement par phrase
- récupérer le doc depuis l'id de la phrase

In [58]:
list(db.collection('tokens').find({'token':"durée"}))

[{'_key': 'token1769',
  '_id': 'tokens/token1769',
  '_rev': '_eMbiN6W-A-',
  'token': 'durée'}]

In [38]:
token_of_interest = 'tokens/token2'

In [51]:
pd.DataFrame(list(db.aql.execute(f'''
                        FOR doc, connection in 1..1 any
                        {'"'+ token_of_interest + '"'} syntagmatic_link
                         return connection
                    ''')))

,_key,_id,_from,_to,_rev,dep_relation,from_sentence_number,head_pos_tag
0,1318149,syntagmatic_link/1318149,tokens/token2,tokens/token3,_eMbiOAu--D,obj,doc0sent0,VERB
1,1318146,syntagmatic_link/1318146,tokens/token0,tokens/token2,_eMbiOAu--A,xcomp,doc0sent0,VERB


## concordancier

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [219]:
words_of_interest = ['point','vue']

In [215]:
def search_tokens(tokens_list,doclist=[]):
    
    tokens_ids=  list(db.aql.execute(f'''let selection =  {tokens_list}
for token in tokens 
filter token.token in selection
return token._id'''))
    
    
    dataframes_list = []
    for word in tokens_ids:
        content = (list(db.aql.execute(f"""for v, e in 1..1
    outbound '{word}'
    is_from
    filter v.content in {tokens_list}
    return v.content""")))

        from_doc = (list(db.aql.execute(f"""for v, e in 2..2
    outbound '{word}'
    is_from
    filter v.content in {tokens_list}
    return v.doc_name""")))

        dataframes_list.append(pd.DataFrame({'sentence':content,
                                             'from_doc':from_doc}))
        
        df_recherche = pd.concat(dataframes_list).drop_duplicates()
        
        sentences_with_comparator = pd.concat([df_recherche,
          pd.DataFrame({'sentence':[' '.join(tokens_list)],
              'from_doc':['comparateur']})]
         ).reset_index(drop=True)
        
        vectorizer = TfidfVectorizer()
        vector_text = vectorizer.fit_transform(sentences_with_comparator['sentence'])
        similarity = cosine_similarity(vector_text[vector_text.shape[0]-1,:],vector_text)
        sentences_with_comparator['similarity'] = similarity[0]
        sentences_with_comparator = sentences_with_comparator.sort_values('similarity', ascending=False)\
                           .reset_index(drop=True)
        final_result = sentences_with_comparator[sentences_with_comparator['from_doc'] != 'comparateur']
    if len(doclist) == 0:
    
        return final_result
    elif len(doclist) > 0: 
        return final_result[final_result['from_doc'].isin(doclist)]

In [217]:
wtv = search_tokens(['groupe'],['girandola 2.txt'])
wtv

,sentence,from_doc,similarity


In [220]:
words_of_interest

In [224]:
pd.Series(["j'ai un autre point de vue"],
           ).str.contains('|'.join(words_of_interest))

0    True
dtype: bool

In [185]:
        wtv[wtv['from_doc'].isin(['lomonaco 1.txt'])]

,sentence,from_doc,similarity
1,Traité de psychologie sociale,lomonaco 1.txt,0.201223
9,Explication causale : un des domaines de prédi...,lomonaco 1.txt,0.106923
15,L’attribution causale se trouve donc être un ...,lomonaco 1.txt,0.088698
22,L’attribution causale se trouve donc au centre...,lomonaco 1.txt,0.074599


# todo :
construire une requette qui récupère seulement les textes où tous les mots sont présents pas seulement un d'entre eux.